In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit,KFold,train_test_split
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor,StackingRegressor
from sklearn.metrics import mean_absolute_error,make_scorer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE,SelectKBest,f_regression
import xgboost as xgb
from hyperopt import fmin, tpe, hp, Trials
from statsmodels.tsa.seasonal import seasonal_decompose
from tqdm import tqdm
pd.set_option('display.max_rows', None)

CSV DOSYALARININ YÜKLENMESİ VE TEST DOSYASININ OLUŞTURULMASI

In [2]:
train = pd.read_csv("data/train.csv")
submission = pd.read_csv("data/sample_submission.csv")

submission['merchant_id'] = submission['id'].astype(str).str[6:]
submission['month_id'] = submission['id'].astype(str).str[:6]
submission.drop(columns=['net_payment_count'],inplace=True)
train.drop(columns=['month_id','net_payment_count'],inplace=True)
test = submission.merge(train,on='merchant_id',how='left').drop_duplicates().reset_index(drop=True)
test.to_csv('data/test.csv',index=False)
test["net_payment_count"] = 0

train = pd.read_csv("data/train.csv")
train.insert(0, "id", train["month_id"].astype(str).str.cat(train["merchant_id"].astype(str)))

train["date"] = pd.to_datetime(train["month_id"], format="%Y%m")
train.sort_values("date",inplace=True)
train = train.reset_index(drop=True)

test["date"] = pd.to_datetime(test["month_id"], format="%Y%m")
test.sort_values("date",inplace=True)
test = test.reset_index(drop=True)



In [3]:
train.head()

,id,merchant_id,month_id,merchant_source_name,settlement_period,working_type,mcc_id,merchant_segment,net_payment_count,date
0,202001merchant_66740,merchant_66740,202001,Merchant Source - 1,Settlement Period - 1,Working Type - 6,mcc_130,Segment - 4,6,2020-01-01
1,202001merchant_12444,merchant_12444,202001,Merchant Source - 1,Settlement Period - 1,Working Type - 2,mcc_153,Segment - 4,3,2020-01-01
2,202001merchant_40154,merchant_40154,202001,Merchant Source - 2,Settlement Period - 1,Working Type - 5,mcc_168,Segment - 4,3,2020-01-01
3,202001merchant_33179,merchant_33179,202001,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_25,Segment - 2,787,2020-01-01
4,202001merchant_16977,merchant_16977,202001,Merchant Source - 2,Settlement Period - 1,Working Type - 6,mcc_31,Segment - 4,5,2020-01-01


In [4]:
train.shape

(291142, 10)

In [5]:
test.head()

,id,merchant_id,month_id,merchant_source_name,settlement_period,working_type,mcc_id,merchant_segment,net_payment_count,date
0,202310merchant_66957,merchant_66957,202310,Merchant Source - 1,Settlement Period - 1,Working Type - 6,mcc_42,Segment - 4,0,2023-10-01
1,202310merchant_23584,merchant_23584,202310,Merchant Source - 1,Settlement Period - 1,Working Type - 5,mcc_40,Segment - 4,0,2023-10-01
2,202310merchant_66460,merchant_66460,202310,Merchant Source - 2,Settlement Period - 1,Working Type - 3,mcc_96,Segment - 4,0,2023-10-01
3,202310merchant_28101,merchant_28101,202310,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_35,Segment - 2,0,2023-10-01
4,202310merchant_36789,merchant_36789,202310,Merchant Source - 1,Settlement Period - 1,Working Type - 6,mcc_126,Segment - 4,0,2023-10-01


In [6]:
test.shape

(78180, 10)